In [1]:
# Create a directory for required files
!mkdir -p opt/ml/model  
!cp model.joblib opt/ml/model/model.joblib
!cp imports_featurizer.pkl opt/ml/model/imports_featurizer.pkl
!cp section_names_featurizer.pkl opt/ml/model/section_names_featurizer.pkl

In [2]:
%%writefile inference.py
import joblib
import os
import json
from scipy.sparse import hstack, csr_matrix
from sklearn.feature_extraction.text import HashingVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
import numpy as np
import pickle
import boto3

def model_fn(model_dir):
    model = joblib.load(os.path.join(model_dir, "model.joblib"))
    return model


def input_fn(request_body, request_content_type):
    if request_content_type == 'application/json':
        input_data = json.loads(request_body)
        return input_data
    else:
        raise ValueError("This model only supports application/json input")


def predict_fn(input_data, model):
    # Process the input data if necessary
    processed_data = process_input(input_data)
    # Make predictions using the model
    predictions = model.predict(processed_data)
    #print(predictions)
    return predictions

def process_input(input_data):
    # Process input data as needed before passing to the model for prediction
    NgramFeaturesList_pred = np.array(input_data['NgramFeaturesList_pred'])
    importsCorpus_pred = input_data['importsCorpus_pred']
    sectionNames_pred = input_data['sectionNames_pred']
    numSections_pred = int(input_data['numSections_pred'])
    

    # Load featurizers
    imports_featurizer = joblib.load(os.path.join("opt/ml/model", "imports_featurizer.pkl"))
    section_names_featurizer = joblib.load(os.path.join("opt/ml/model", "section_names_featurizer.pkl"))
    #print(NgramFeaturesList_pred, importsCorpus_pred, sectionNames_pred, numSections_pred)
    #print(imports_featurizer, section_names_featurizer)
    # Transform text features
    importsCorpus_pred_transformed = imports_featurizer.transform([importsCorpus_pred])
    sectionNames_pred_transformed = section_names_featurizer.transform([sectionNames_pred])

    # Concatenate features into a single sparse matrix
    processed_data = hstack([csr_matrix(NgramFeaturesList_pred),
                             importsCorpus_pred_transformed,
                             sectionNames_pred_transformed,
                             csr_matrix([numSections_pred]).transpose()])
    #print(processed_data)
    return processed_data


def output_fn(prediction, content_type):
    res = int(prediction[0])
    respJSON = {'Output': res}
    return respJSON

Writing inference.py


In [3]:
%%writefile requirements.txt
joblib
scipy
numpy
scikit-learn

Writing requirements.txt


In [6]:
# Tarballing the required files
!tar -cvpzf modeldeployment.tar.gz model.joblib inference.py imports_featurizer.pkl section_names_featurizer.pkl requirements.txt opt

model.joblib
inference.py
imports_featurizer.pkl
section_names_featurizer.pkl
requirements.txt
opt/
opt/ml/
opt/ml/model/
opt/ml/model/model.joblib
opt/ml/model/section_names_featurizer.pkl
opt/ml/model/imports_featurizer.pkl
opt/ml/model/.ipynb_checkpoints/


In [7]:
# Importing libraries
import boto3
import json
import os
import joblib
import pickle
import tarfile
import sagemaker
from sagemaker.estimator import Estimator
import time
from time import gmtime, strftime
import subprocess

# Setting up Boto3
client = boto3.client(service_name="sagemaker")
runtime = boto3.client(service_name="sagemaker-runtime")
boto_session = boto3.session.Session()
s3 = boto_session.resource('s3')
region = boto_session.region_name
print(region)
sagemaker_session = sagemaker.Session()
role = "arn:aws:iam::804624801206:role/LabRole"

# Retrieving SKLearn image URI
image_uri = sagemaker.image_uris.retrieve(
    framework="sklearn",
    region=region,
    version="1.2-1",
    py_version="py3",
    instance_type="ml.t2.medium",
)

# Specifying the S3 bucket
default_bucket = sagemaker_session.default_bucket()

# Uploading the model tarball to S3
model_artifacts = f"s3://{default_bucket}/modeldeployment.tar.gz"
response = s3.meta.client.upload_file('modeldeployment.tar.gz', default_bucket, 'modeldeployment.tar.gz')

# Creating the model
model_name = "sklearn-test" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
create_model_response = client.create_model(
    ModelName=model_name,
    Containers=[
        {
            "Image": image_uri,
            "Mode": "SingleModel",
            "ModelDataUrl": model_artifacts,
            "Environment": {'SAGEMAKER_SUBMIT_DIRECTORY': model_artifacts,
                           'SAGEMAKER_PROGRAM': 'inference.py'} 
        }
    ],
    ExecutionRoleArn=role,
)
print("Model Arn: " + create_model_response["ModelArn"])

# Creating the endpoint configuration
sklearn_epc_name = "sklearn-epc" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
endpoint_config_response = client.create_endpoint_config(
    EndpointConfigName=sklearn_epc_name,
    ProductionVariants=[
        {
            "VariantName": "sklearnvariant",
            "ModelName": model_name,
            "InstanceType": "ml.t2.medium",
            "InitialInstanceCount": 1
        },
    ],
)
print("Endpoint Configuration Arn: " + endpoint_config_response["EndpointConfigArn"])

# Creating the endpoint
endpoint_name = "sklearn-local-ep" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
create_endpoint_response = client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=sklearn_epc_name,
)
print("Endpoint Arn: " + create_endpoint_response["EndpointArn"])

# Monitoring the creation
describe_endpoint_response = client.describe_endpoint(EndpointName=endpoint_name)
while describe_endpoint_response["EndpointStatus"] == "Creating":
    describe_endpoint_response = client.describe_endpoint(EndpointName=endpoint_name)
    print(describe_endpoint_response["EndpointStatus"])
    time.sleep(5)
print(describe_endpoint_response)

us-east-1
Model Arn: arn:aws:sagemaker:us-east-1:804624801206:model/sklearn-test2024-04-04-22-54-36
Endpoint Configuration Arn: arn:aws:sagemaker:us-east-1:804624801206:endpoint-config/sklearn-epc2024-04-04-22-54-37
Endpoint Arn: arn:aws:sagemaker:us-east-1:804624801206:endpoint/sklearn-local-ep2024-04-04-22-54-37
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating


In [9]:
# Testing the endpoint
import json
import boto3

# Initialize the SageMaker runtime client
runtime = boto3.client('sagemaker-runtime')

# Define your input data
input_data = {
    'NgramFeaturesList_pred': [[24183, 3382, 304, 17, 923, 636, 358, 275, 128, 635, 358, 613, 389, 384, 448, 12, 380, 170, 307, 122, 224, 203, 51, 338, 521, 111, 395, 215, 175, 419, 264, 397, 287, 106, 487, 236, 16, 277, 459, 594, 469, 241, 155, 163, 158, 230, 215, 443, 80, 46, 44, 216, 68, 42, 36, 48, 161, 29, 240, 145, 139, 52, 20, 75, 99, 33, 224, 161, 38, 226, 729, 139, 27, 168, 19, 68, 269, 271, 236, 33, 197, 207, 337, 1114, 126, 111, 255, 175, 47, 46, 60, 318, 129, 79, 16, 223, 162, 79, 15, 157]],
    'importsCorpus_pred': "kernel32 shlwapi ole32 shell32 user32",
    'sectionNames_pred': ".text .rdata .data .rsrc .reloc",
    'numSections_pred': "5"
}

# Convert input data to JSON string
payload = json.dumps(input_data)

# Specify the endpoint name
endpoint_name = 'sklearn-local-ep2024-04-04-22-54-37'

# Call the endpoint
response = runtime.invoke_endpoint(EndpointName=endpoint_name,
                                   ContentType='application/json',
                                   Body=payload)

# Decode and print the response
result = json.loads(response['Body'].read().decode())
print(result)


{'Output': 1}
